In [2]:
import openai
import os

# Set up the OpenAI API credentials, replace this with your key
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxx"

In [3]:
sql_schmea = """
  CREATE TABLE orders (
    ORDERNUMBER INT,
    QUANTITYORDERED INT,
    PRICEEACH DECIMAL(10,2),
    ORDERLINENUMBER INT,
    SALES DECIMAL(10,2),
    ORDERDATE DATE,
    STATUS VARCHAR(50),
    QTR_ID INT,
    MONTH_ID INT,
    YEAR_ID INT,
    PRODUCTLINE VARCHAR(50),
    MSRP DECIMAL(10,2),
    PRODUCTCODE VARCHAR(50),
    CUSTOMERNAME VARCHAR(100),
    PHONE VARCHAR(50),
    ADDRESSLINE1 VARCHAR(100),
    ADDRESSLINE2 VARCHAR(100),
    CITY VARCHAR(50),
    STATE VARCHAR(50),
    POSTALCODE VARCHAR(20),
    COUNTRY VARCHAR(50),
    TERRITORY VARCHAR(50),
    CONTACTLASTNAME VARCHAR(50),
    CONTACTFIRSTNAME VARCHAR(50),
    DEALSIZE VARCHAR(50)
  );
  """


import psycopg2
import pandas as pd

def get_order_data(sql):

    conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="password", port="5432")
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    cur.close()
    conn.close()
    df = pd.DataFrame(rows)    
    return df


In [4]:

def ask_gpt_and_run_query(question):
  # Define the prompt and parameters for the API request
  prompt = question + """
    Given the schema below on a postgres database: 
  """ + sql_schmea

  model = "text-davinci-002"
  temperature = 0
  max_tokens = 100

  # Call the API to generate text
  response = openai.Completion.create(
      engine=model,
      prompt=prompt,
      temperature=temperature,
      max_tokens=max_tokens
  )
  print (f'SQL : {response.choices[0].text}')
  print (f'------------------------')
  print (get_order_data(response.choices[0].text))


In [94]:
ask_gpt_and_run_query("Get the total sales for each product line for the year 2003.") 


SQL : 
  SELECT productline, sum(sales)
  FROM orders
  WHERE year_id = 2003
  GROUP BY productline;
------------------------
                  0           1
0      Classic Cars  1484785.29
1            Trains    72802.29
2            Planes   272257.60
3  Trucks and Buses   420429.93
4      Vintage Cars   650987.76
5       Motorcycles   370895.58
6             Ships   244821.09


In [95]:
ask_gpt_and_run_query("Get the top 10 customers who gave the highest sales for the year 2003.")

SQL : 
  SELECT customerName, SUM(sales) AS total_sales
  FROM orders
  WHERE year_id = 2003
  GROUP BY customerName
  ORDER BY total_sales DESC
  LIMIT 10;
------------------------
                              0          1
0         Euro Shopping Channel  210227.58
1  Mini Gifts Distributors Ltd.  185128.12
2       Dragon Souveniers, Ltd.  165686.20
3            Muscle Machine Inc  132778.24
4          Technics Stores Inc.  104337.30
5           Mini Creations Ltd.   97929.83
6                 Rovelli Gifts   96259.03
7      Corporate Gift Ideas Co.   95678.88
8                  Herkku Gifts   95277.18
9       Anna's Decorations, Ltd   88983.71


In [100]:
ask_gpt_and_run_query("What was the order number and the client name of the highest sales?  Run SQL")

SQL : 

SELECT ORDERNUMBER, CUSTOMERNAME, SALES
FROM orders
ORDER BY SALES DESC
LIMIT 1;
------------------------
       0                          1         2
0  10407  The Sharp Gifts Warehouse  14082.80


In [103]:
ask_gpt_and_run_query("How many records are there in the orders table?  Give me the SQL to get the number of records in the orders table.")

SQL : 
  SELECT COUNT(*) FROM orders;
------------------------
      0
0  2823
